In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
import re

In [4]:
os.chdir('/Users/user/Desktop/ВКР/')

names = glob.glob('*.csv')
names

['processed_IMDB_dataset.csv',
 'df_films_from_data.csv',
 'processed_russian_dataset.csv']

In [33]:
df = pd.read_csv('processed_russian_dataset.csv')

In [34]:
X = df.drop(['Unnamed: 0','ranking', 'index', 'english_name', 'film_id', 'budget', 'currency', 'film_age', 'actors', 'directors', 'currency_course_to_dollars'], axis = 1)

In [35]:
X['countries'] = X['countries'].apply(lambda x: eval(x)[0] if eval(x) else 'other')
X['companies'] = X['companies'].apply(lambda x: eval(x)[0] if eval(x) else 'other')
X['genres'] = X['genres'].apply(lambda x: eval(x))

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
res = mlb.fit_transform(X['genres'])
for i in range(len(mlb.classes_)):
    X[mlb.classes_[i]] = res.T[i]

X = X.drop(['genres'], axis = 1)

In [37]:
X = X.rename(columns={'budget_in_dollars': 'budget', 
                      'companies': 'production_companies', 
                      'countries': 'production_countries', 
                     'rating_await': 'vote_count', 
                     'film_length': 'runtime', 
                     'actors_ranking': 'cast_ranking'})

In [38]:
all_genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'Thriller', 'War', 'Western', 'Documentary', 'Foreign', 'TV Movie']

In [39]:
for g in all_genres:
    if g not in X.columns:
        X[g] = np.zeros(len(X)).astype(int)

In [44]:
X.columns

Index(['belongs_to_collection', 'production_countries', 'production_companies',
       'runtime', 'vote_count', 'budget', 'cast_ranking', 'crew_ranking',
       'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'Thriller', 'War', 'Western', 'Documentary',
       'Foreign', 'TV Movie'],
      dtype='object')

In [45]:
X = X[['belongs_to_collection', 'budget', 'production_companies', 'production_countries', 'runtime', 'vote_count', 
      'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western', 'cast_ranking', 'crew_ranking']]

In [46]:
import joblib
cbr = joblib.load("model.pkl")

In [49]:
predict = cbr.predict(X)

In [50]:
df['predicted_box_office'] = predict

In [51]:
df.to_csv('russian_dataset_with_predicted_box_office', index = False)